In [4]:
import tensorflow as tf
from tensorflow.python import keras

In [5]:
print(keras.__version__)

2.6.0


In [20]:
model = keras.models.load_model("/data/study/git/pytorch_cook/04_trt/data/b1.h5")

In [23]:
import numpy as np
from skimage import io
from skimage.transform import resize
# from matplotlib import pyplot as plt
BATCH_SIZE = 1024
url='06l2_test.png'
img = resize(io.imread(url), (96, 62, 1))
input_batch = 255*np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)

input_batch.shape

(1024, 96, 62, 1)

In [24]:
predictions = model.predict(input_batch) # warm up
indices = (-predictions[0]).argsort()[:5]
print("Class | Probability (out of 1)")
list(zip(indices, predictions[0][indices]))



2022-06-09 09:23:12.639252: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-06-09 09:23:13.749048: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-09 09:23:15.111926: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Class | Probability (out of 1)


[(13, 0.98952967),
 (3, 0.008999778),
 (0, 0.00051976135),
 (17, 0.0002356667),
 (9, 0.00013105824)]

In [25]:
%%timeit

result = model.predict_on_batch(input_batch) # Check default performance

151 ms ± 6.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
import onnx

In [27]:
model.save('my_model')
!python -m tf2onnx.convert --saved-model my_model --output temp.onnx
onnx_model = onnx.load_model('temp.onnx')

2022-06-09 09:24:00.557980: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: my_model/assets


/root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


/root/miniconda3/envs/tf26_trt/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-06-09 09:24:26,730 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-06-09 09:24:27,150 - WARNING - Importing a function (__inference_block5b_se_reduce_layer_call_and_return_conditional_losses_31609) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2022-06-09 09:24:27,157 - WARNING - Importing a function (__inference_block6a_expand_activation_layer_call_and_return_conditional_losses_62030) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2022-06-09 09:24:27,187 - WARNING - Importing a function (__inference_block6b_activation_layer_call_and_return_conditional_losses_62675) with ops with unsaved custom gradients. Will likely fai

In [28]:
inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = BATCH_SIZE

In [29]:
model_name = "resnet50_onnx_model_tf.onnx"
onnx.save_model(onnx_model, model_name)
print("Done saving!")

Done saving!


In [30]:
import os, time
print("Restarting kernel  in three seconds...")
time.sleep(3)
print("Restarting kernel now")
os._exit(0) # Shut down all kernels so TRT doesn't fight with Tensorflow fo

Restarting kernel  in three seconds...


: 

: 

In [2]:
BATCH_SIZE = 32

In [4]:
import numpy as np

USE_FP16 = False

target_dtype = np.float16 if USE_FP16 else np.float32

In [5]:
import numpy as np
from skimage import io
from skimage.transform import resize
# from matplotlib import pyplot as plt
BATCH_SIZE = 1024
url='06l2_test.png'
img = resize(io.imread(url), (96, 62, 1))
input_batch = 255*np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)

input_batch.shape

(1024, 96, 62, 1)

In [6]:
input_batch = input_batch.astype(target_dtype)

In [7]:
# May need to shut down all kernels and restart before this - otherwise you might get cuDNN initialization errors:
if USE_FP16:
    !trtexec --onnx=resnet50_onnx_model_tf.onnx --saveEngine=resnet_engine.trt  --explicitBatch --inputIOFormats=fp16:chw --outputIOFormats=fp16:chw --fp16
else:
    !trtexec --onnx=resnet50_onnx_model_tf.onnx --saveEngine=resnet_engine.trt  --explicitBatch

&&&& RUNNING TensorRT.trtexec [TensorRT v8204] # trtexec --onnx=resnet50_onnx_model_tf.onnx --saveEngine=resnet_engine.trt --explicitBatch
[06/09/2022-09:26:26] [W] --explicitBatch flag has been deprecated and has no effect!
[06/09/2022-09:26:26] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[06/09/2022-09:26:26] [I] === Model Options ===
[06/09/2022-09:26:26] [I] Format: ONNX
[06/09/2022-09:26:26] [I] Model: resnet50_onnx_model_tf.onnx
[06/09/2022-09:26:26] [I] Output:
[06/09/2022-09:26:26] [I] === Build Options ===
[06/09/2022-09:26:26] [I] Max batch: explicit batch
[06/09/2022-09:26:26] [I] Workspace: 16 MiB
[06/09/2022-09:26:26] [I] minTiming: 1
[06/09/2022-09:26:26] [I] avgTiming: 8
[06/09/2022-09:26:26] [I] Precision: FP32
[06/09/2022-09:26:26] [I] Calibration: 
[06/09/2022-09:26:26] [I] Refit: Disabled
[06/09/2022-09:26:26] [I] Sparsity: Disabled
[06/09/2022-09:26:26] [I] Safe mode: Disabled
[06

In [8]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

f = open("resnet_engine.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING)) 

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

[06/09/2022-09:42:36] [TRT] [W] TensorRT was linked against cuBLAS/cuBLASLt 11.6.5 but loaded cuBLAS/cuBLASLt 11.5.2
[06/09/2022-09:42:37] [TRT] [W] TensorRT was linked against cuBLAS/cuBLASLt 11.6.5 but loaded cuBLAS/cuBLASLt 11.5.2


In [13]:
output = np.empty([BATCH_SIZE, 22], dtype = target_dtype) # Need to set output dtype to FP16 to enable FP16

# Allocate device memory
d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

In [14]:
def predict(batch): # result gets copied into output
    # Transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # Execute model
    context.execute_async_v2(bindings, stream.handle, None)
    # Transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # Syncronize threads
    stream.synchronize()
    
    return output

In [15]:
print("Warming up...")

trt_predictions = predict(input_batch).astype(np.float32)

print("Done warming up!")

Warming up...
Done warming up!


In [16]:
indices = (-trt_predictions[0]).argsort()[:5]
print("Class | Probability (out of 1)")
list(zip(indices, trt_predictions[0][indices]))

Class | Probability (out of 1)


[(13, 0.9894561),
 (3, 0.009055261),
 (0, 0.0005255092),
 (17, 0.00023879894),
 (9, 0.00013224599)]

In [18]:
%%timeit

_ = predict(input_batch) # Check TRT performance

46.5 ms ± 31.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit

result = model.predict_on_batch(input_batch) # Check default performance